<a href="https://colab.research.google.com/github/abhy-kumar/wind-potential/blob/main/osw_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas geopandas folium numpy shapely branca

In [ ]:
import requests
import pandas as pd
import folium
import numpy as np
from branca.colormap import LinearColormap
from datetime import datetime, timedelta
import json

def fetch_gfs_wind_data(lat, lon):
    """
    Fetch wind data from NOAA GFS API
    Returns U and V wind components at 100m height
    """
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "YOUR_API_KEY"  # You'll need to sign up for a free API key at OpenWeatherMap

    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data['wind']['speed']
        return None
    except Exception as e:
        print(f"Error fetching data for lat: {lat}, lon: {lon}")
        print(f"Error details: {str(e)}")
        return None

def calculate_wind_power_density(wind_speed):
    """
    Calculate wind power density in W/m²
    Using standard air density at sea level (1.225 kg/m³)
    """
    if wind_speed is None:
        return None
    air_density = 1.225  # kg/m³
    return 0.5 * air_density * (wind_speed ** 3)

def generate_sample_wind_data():
    """
    Generate sample wind data based on typical wind patterns along Indian coast
    This is used when API access is not available
    """
    # Define detailed coastal regions with their characteristics
    coastal_regions = {
        'Gujarat': {
            'start': [(23.3, 68.5)],  # Kutch
            'end': [(20.3, 72.8)],    # Daman
            'base_speed': 8.5,         # Higher wind speeds in Gujarat
            'seasonal_variation': 2.0
        },
        'Maharashtra': {
            'start': [(20.3, 72.8)],
            'end': [(15.7, 73.8)],
            'base_speed': 7.5,
            'seasonal_variation': 1.8
        },
        'Goa': {
            'start': [(15.7, 73.8)],
            'end': [(15.0, 74.0)],
            'base_speed': 7.0,
            'seasonal_variation': 1.5
        },
        'Karnataka': {
            'start': [(15.0, 74.0)],
            'end': [(12.4, 74.9)],
            'base_speed': 6.8,
            'seasonal_variation': 1.5
        },
        'Kerala': {
            'start': [(12.4, 74.9)],
            'end': [(8.1, 77.2)],
            'base_speed': 6.5,
            'seasonal_variation': 2.0
        },
        'Tamil Nadu': {
            'start': [(8.1, 77.2)],
            'end': [(13.5, 80.3)],
            'base_speed': 7.0,
            'seasonal_variation': 2.5  # Higher variation due to monsoons
        },
        'Andhra Pradesh': {
            'start': [(13.5, 80.3)],
            'end': [(19.1, 84.8)],
            'base_speed': 7.2,
            'seasonal_variation': 2.0
        },
        'Odisha': {
            'start': [(19.1, 84.8)],
            'end': [(21.6, 87.5)],
            'base_speed': 7.0,
            'seasonal_variation': 1.8
        },
        'West Bengal': {
            'start': [(21.6, 87.5)],
            'end': [(21.9, 89.0)],    # Sundarbans
            'base_speed': 6.8,
            'seasonal_variation': 1.5
        }
    }

    data = []

    # Generate detailed points along the coastline
    for region, info in coastal_regions.items():
        for start_point, end_point in zip(info['start'], info['end']):
            # Calculate the number of points based on the greater distance
            lat_diff = abs(end_point[0] - start_point[0])
            lon_diff = abs(end_point[1] - start_point[1])
            num_points = int(max(lat_diff, lon_diff) * 10) + 1  # 0.1 degree intervals

            # Create points along the coast
            lats = np.linspace(start_point[0], end_point[0], num_points)
            lons = np.linspace(start_point[1], end_point[1], num_points)

            # Generate points following the coastline
            for lat, lon in zip(lats, lons):

                # Generate realistic wind speeds based on location and regional characteristics
                base_speed = info['base_speed']

                # Add seasonal and local variations
                seasonal_factor = np.sin(2 * np.pi * (datetime.now().month / 12)) * info['seasonal_variation']
                local_variation = np.random.normal(0, 0.5)  # Local geographic variations

                # Calculate final wind speed
                wind_speed = base_speed + seasonal_factor + local_variation

                # Add offshore boost (wind speeds typically increase offshore)
                offshore_boost = np.random.uniform(0.5, 1.5)
                wind_speed += offshore_boost

                # Keep within realistic bounds
                wind_speed = max(4, min(14, wind_speed))

                # Calculate power density
                power_density = calculate_wind_power_density(wind_speed)

                # Add point to dataset
                data.append({
                    'latitude': lat,
                    'longitude': lon,
                    'wind_speed': wind_speed,
                    'power_density': power_density,
                    'region': region
                })

                # Add offshore points (20km and 40km from coast)
                for distance in [0.2, 0.4]:  # Approximate degree offsets for 20km and 40km
                    if region in ['Gujarat', 'Maharashtra', 'Goa', 'Karnataka', 'Kerala']:
                        # West coast - add points to the west
                        offshore_lon = lon - distance
                    else:
                        # East coast - add points to the east
                        offshore_lon = lon + distance

                    # Increase wind speed for offshore points
                    offshore_wind = wind_speed + distance * 2 + np.random.normal(0, 0.3)
                    offshore_wind = max(5, min(15, offshore_wind))

                    data.append({
                        'latitude': lat,
                        'longitude': offshore_lon,
                        'wind_speed': offshore_wind,
                        'power_density': calculate_wind_power_density(offshore_wind),
                        'region': f"{region} (Offshore)"
                    })

    return pd.DataFrame(data)

def create_wind_potential_map():
    print("Generating wind data...")

    # Use sample data since we can't access the API without a key
    df = generate_sample_wind_data()

    print(f"Creating map with {len(df)} data points...")

    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Average Wind Speed: {df['wind_speed'].mean():.2f} m/s")
    print(f"Average Power Density: {df['power_density'].mean():.2f} W/m²")

    # Create map centered on India
    m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

    # Create color map
    colormap = LinearColormap(
        colors=['yellow', 'orange', 'red'],
        vmin=df['power_density'].min(),
        vmax=df['power_density'].max(),
        caption='Wind Power Density (W/m²)'
    )

    # Add points to map with region information
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=10,
            color=None,
            fill=True,
            fill_color=colormap(row['power_density']),
            fill_opacity=0.7,
            popup=f"Region: {row['region']}<br>"
                  f"Wind Speed: {row['wind_speed']:.1f} m/s<br>"
                  f"Power Density: {row['power_density']:.0f} W/m²"
        ).add_to(m)

    # Add colormap to map
    colormap.add_to(m)

    # Add title
    title_html = '''
        <div style="position: fixed;
                    top: 10px; left: 50px; width: 300px; height: 30px;
                    z-index:9999; font-size:16px; background-color: white;
                    border-radius: 5px; padding: 10px">
            <b>Indian Coastal Wind Power Potential</b>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))

    # Save map
    output_file = 'india_offshore_wind_potential.html'
    m.save(output_file)
    print(f"\nMap saved as '{output_file}'")

    # Save data to CSV for reference
    df.to_csv('wind_potential_data.csv', index=False)
    print("Data saved to 'wind_potential_data.csv'")

if __name__ == "__main__":
    create_wind_potential_map()

Generating wind data...
Creating map with 969 data points...

Summary Statistics:
Average Wind Speed: 9.61 m/s
Average Power Density: 559.43 W/m²

Map saved as 'india_offshore_wind_potential.html'
Data saved to 'wind_potential_data.csv'


In [ ]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap
from datetime import datetime
import branca.colormap as cm
from scipy.interpolate import griddata

def generate_coastal_points():
    """
    Generate points along the Indian coastline with better accuracy
    """
    # Define more detailed coastal coordinates
    coastal_coordinates = {
        'Gujarat': [
            (23.3, 68.5), (22.8, 69.7), (22.5, 69.9),  # Kutch
            (22.2, 69.1), (21.6, 72.3), (20.3, 72.8)   # Gujarat coast
        ],
        'Maharashtra': [
            (20.3, 72.8), (19.9, 72.8), (19.1, 72.9),
            (18.4, 72.8), (17.3, 73.3), (15.7, 73.8)
        ],
        'Goa': [
            (15.7, 73.8), (15.4, 73.8), (15.0, 74.0)
        ],
        'Karnataka': [
            (15.0, 74.0), (14.5, 74.3), (13.5, 74.7),
            (12.8, 74.8), (12.4, 74.9)
        ],
        'Kerala': [
            (12.4, 74.9), (11.8, 75.4), (10.9, 75.8),
            (9.9, 76.2), (8.9, 76.6), (8.1, 77.2)
        ],
        'Tamil Nadu': [
            (8.1, 77.2), (8.6, 78.1), (9.3, 79.0),
            (10.3, 79.8), (11.3, 79.8), (12.3, 80.0),
            (13.5, 80.3)
        ],
        'Andhra Pradesh': [
            (13.5, 80.3), (14.5, 80.2), (15.5, 80.8),
            (16.5, 82.2), (17.7, 83.3), (18.4, 84.1),
            (19.1, 84.8)
        ],
        'Odisha': [
            (19.1, 84.8), (19.9, 85.8), (20.3, 86.5),
            (20.8, 87.0), (21.6, 87.5)
        ],
        'West Bengal': [
            (21.6, 87.5), (21.8, 88.1), (21.9, 88.4),
            (21.9, 89.0)
        ]
    }

    points = []
    for region, coords in coastal_coordinates.items():
        for i in range(len(coords) - 1):
            # Generate points between each pair of coordinates
            start = coords[i]
            end = coords[i + 1]

            # Create more points for longer segments
            distance = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            num_points = max(int(distance * 20), 5)  # Adjust density of points

            lats = np.linspace(start[0], end[0], num_points)
            lons = np.linspace(start[1], end[1], num_points)

            for lat, lon in zip(lats, lons):
                # Generate offshore points perpendicular to coastline
                angle = np.arctan2(end[1] - start[1], end[0] - start[0]) + np.pi/2
                for dist in np.linspace(0, 0.5, 5):  # 0 to ~50km offshore
                    off_lat = lat + dist * np.cos(angle)
                    off_lon = lon + dist * np.sin(angle)
                    points.append((off_lat, off_lon, region))

    return points

def calculate_wind_power(lat, base_speed=7.0):
    """
    Calculate wind power based on latitude and typical patterns
    """
    # Latitude factor (higher winds at higher latitudes)
    lat_factor = 0.2 * (lat - 8.0)

    # Seasonal variation (using current month)
    month = datetime.now().month
    seasonal_factor = 2 * np.sin(2 * np.pi * (month - 1) / 12)

    # Offshore gradient (increases with distance from coast)
    base_speed = base_speed + lat_factor + seasonal_factor

    # Add some random variation
    variation = np.random.normal(0, 0.5)
    wind_speed = base_speed + variation

    # Calculate power density (W/m²)
    air_density = 1.225  # kg/m³
    return 0.5 * air_density * (wind_speed ** 3)

def create_wind_potential_map():
    print("Generating coastal points...")
    coastal_points = generate_coastal_points()

    # Create DataFrame with wind power data
    data = []
    for lat, lon, region in coastal_points:
        power = calculate_wind_power(lat)
        data.append({
            'latitude': lat,
            'longitude': lon,
            'power_density': power,
            'region': region
        })

    df = pd.DataFrame(data)

    print("Creating heatmap...")

    # Create base map
    m = folium.Map(
        location=[20.5937, 78.9629],
        zoom_start=5,
        tiles='cartodbpositron'
    )

    # Prepare heatmap data
    heat_data = df[['latitude', 'longitude', 'power_density']].values.tolist()

    # Create custom colormap
    gradient = {
        0.2: '#fee090',  # Light yellow
        0.4: '#fdae61',  # Light orange
        0.6: '#f46d43',  # Dark orange
        0.8: '#d73027',  # Red
        1.0: '#a50026'   # Dark red
    }

    # Add heatmap layer
    HeatMap(
        heat_data,
        min_opacity=0.4,
        max_opacity=0.8,
        radius=15,
        blur=20,
        gradient=gradient
    ).add_to(m)

    # Add colormap
    colormap = cm.LinearColormap(
        colors=list(gradient.values()),
        vmin=df['power_density'].min(),
        vmax=df['power_density'].max(),
        caption='Wind Power Density (W/m²)'
    )
    colormap.add_to(m)

    # Add title
    title_html = '''
        <div style="position: fixed;
                    top: 10px; left: 50px; width: 300px;
                    z-index:9999; font-size:16px;
                    background-color: white;
                    border-radius: 5px; padding: 10px;
                    box-shadow: 0 0 15px rgba(0,0,0,0.2);">
            <b>Indian Coastal Wind Power Potential</b>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))

    # Save map
    output_file = 'india_offshore_wind_potential.html'
    m.save(output_file)
    print(f"\nMap saved as '{output_file}'")

if __name__ == "__main__":
    create_wind_potential_map()

Generating coastal points...
Creating heatmap...


AttributeError: 'float' object has no attribute 'split'